In [2]:
import requests 
import time
import json
from bs4 import  BeautifulSoup
import pandas as pd

In [2]:
with open('place_info_contain_location.txt', 'r') as f:   
    lines = f.readlines() 
lines_str = str(lines).replace('\'', '"')[2:-2]
city_url_info = json.loads(lines_str)

In [3]:
'''
    获取区域的所有景点
'''
def get_place_url(url):
    response = requests.get(url)
    location_place_url_soup=BeautifulSoup(response.text,'lxml')
    location_place_url_data = location_place_url_soup.select('#tiles > li')

    place_info_list = []

    for item in location_place_url_data:

        try:
            place_info={}
            place_info['href'] = item.a.get('href')

            item2 = item.select('li > div.bar')[0]
            place_info['name'] = item2.attrs['m']
            place_info['ever'] = item2.select('a.ever > span')[0].get_text()
            place_info['want'] = item2.select('a.never > span')[0].get_text()

            place_info_list.append(place_info)

        except Exception as e:
            print(e)
    time.sleep(2)
    return place_info_list

In [4]:
'''
    获取景点信息
'''
def get_place_info(url):

    place_response = requests.get(url)
    place_soup=BeautifulSoup(place_response.text,'html5lib')
    place_header_data = place_soup.select('#jdleft > div.info > div > div.hd > ul > li')
    place_data = place_soup.select('#jdleft > div.info > div > div > div.item')

    place_info_item = {}
    for i in range(len(place_header_data)):

        if place_header_data[i].get_text() in ['学校介绍', '景区介绍', '旅游攻略', '怎么去', '门票']:
            # print(place_header_data[i].get_text())
            # print(place_data[i])
            item_header = place_data[i].find_all('h2')
            # print(item_header)
            place_data_text = place_data[i].get_text()
            
            if place_header_data[i].get_text()=='旅游攻略':
                lv_gl_text_data = place_data[i].select('ul')
                if len(lv_gl_text_data) > 0:
                    lv_gl_text = lv_gl_text_data[0].get_text()
                    place_data_text = place_data_text.replace(lv_gl_text, '')
            
            place_data_text = place_data_text.replace('信息纠错', '')
            place_data_text = place_data_text.replace('景区的卫星地图', '')

            

            place_info_item[place_header_data[i].get_text()] = {}

            for j in range(len(item_header)):
                current_header = item_header[j].get_text()[:-1]
                current_text = place_data_text[place_data_text.find(current_header)+len(current_header)+1:]

                if j < len(item_header)-1:
                    next_header = item_header[j+1].get_text()
                    current_text = current_text[:current_text.find(next_header)]
                
                if current_header not in ['景区游览图', '学校地图', '景区地图', '驴友游记']:
                    place_info_item[place_header_data[i].get_text()][current_header] = current_text.replace('　　', '').replace(' ', '').split('\n')
    time.sleep(2)
    
    return place_info_item



In [5]:
location_error_list = []
error_list = []
base_url = 'https://www.meet99.com'
rlt = {'name':[], 'province': [], 'city': [], 'location': [], 'desc': [], 'want': [], 'has_been': [], 'strategy': [], 'route': [], 'ticket_info': [], 'open_time': []}

for province_key in city_url_info.keys():
    if province_key in ['广东省']:
        for city_key in city_url_info[province_key]['children'].keys():
            # if city_key in ['福州市', '厦门市']:
            #     continue
            # print(city_key)
            
            if 'children' in city_url_info[province_key]['children'][city_key] and len(city_url_info[province_key]['children'][city_key]['children'].keys()) > 0 :
                
                for location_key in city_url_info[province_key]['children'][city_key]['children'].keys():
                    location_info = city_url_info[province_key]['children'][city_key]['children'][location_key]
                    location_url = base_url + location_info['link']
                    print(location_key)
                    print(location_url)
                    location_place_info_list = []
                    try:
                        location_place_info_list = get_place_url(location_url)

                    except Exception as e:
                        print('获取区域内的景点出现异常,异常原因:', e)
                        print(location_info)
                        location_error_info = {}
                        location_error_info['province_key'] = province_key
                        location_error_info['city_key'] = city_key
                        location_error_info['location_name'] = location_key
                        location_error_info['location_info'] = location_info
                        location_error_list.append(location_error_info)


                    for place_item in location_place_info_list:
                        try:
                            print(place_item)
                            place_url = base_url + place_item['href']
                            place_info = get_place_info(place_url)

                            name = place_item['name']
                            
                            province = province_key
                            city = city_key
                            location = location_key
                            desc = {}
                            if '景区介绍' in place_info.keys():
                                desc = place_info['景区介绍']
                            elif '学校介绍' in place_info.keys():
                                desc = place_info['学校介绍']

                            want = place_item['want']
                            has_been = place_item['ever']

                            strategy = {}
                            if '旅游攻略' in place_info.keys():
                                strategy = place_info['旅游攻略']

                            route = {}
                            if '怎么去' in place_info.keys():
                                route = place_info['怎么去']

                            ticket_info = {}
                            open_time = {}
                            if '门票' in place_info.keys():
                                for key in place_info['门票']:
                                    if key not in ['景区开放时间']:
                                        ticket_info[key] = place_info['门票'][key]


                                if '景区开放时间' in place_info['门票'].keys():
                                    open_time = place_info['门票']['景区开放时间']
                            
                            ticket = ticket_info

                            rlt['name'].append(name)
                            rlt['province'].append(province)
                            rlt['city'].append(city)
                            rlt['location'].append(location)
                            rlt['desc'].append(desc)
                            rlt['want'].append(want)
                            rlt['has_been'].append(has_been)
                            rlt['strategy'].append(strategy)
                            rlt['route'].append(route)
                            rlt['ticket_info'].append(ticket_info)
                            rlt['open_time'].append(open_time)
                            

                        except Exception as e:
                            print('以下景点出现异常,异常原因:', e)
                            print(place_item)
                            error_info = {}
                            error_info['province_key'] = province_key
                            error_info['city_key'] = city_key
                            error_info['location_name'] = location_key
                            error_info['place_info'] = place_item
                            error_list.append(error_info)  

                time.sleep(2)

                    


荔湾区
https://www.meet99.com/lvyou-liwan.html
'NoneType' object has no attribute 'get'
{'href': '/jingdian-chenjiaci.html', 'name': '广东陈家祠博物馆', 'ever': '1307', 'want': '164'}
{'href': '/jingdian-shangxiajiu.html', 'name': '广州上下九步行街', 'ever': '888', 'want': '118'}
{'href': '/jingdian-shamian.html', 'name': '广州沙面大街', 'ever': '694', 'want': '81'}
{'href': '/jingdian-liwanbowuguan.html', 'name': '广州荔湾区博物馆', 'ever': '32', 'want': '3'}
{'href': '/jingdian-yongqingfang.html', 'name': '广州永庆坊非遗街区', 'ever': '13', 'want': '1'}
越秀区
https://www.meet99.com/lvyou-yuexiu.html
'NoneType' object has no attribute 'get'
{'href': '/jingdian-guangzhoudongwuyuan.html', 'name': '广州动物园', 'ever': '4261', 'want': '1123'}
{'href': '/jingdian-zhongshanjiniantang.html', 'name': '广州中山纪念堂', 'ever': '4520', 'want': '684'}
{'href': '/jingdian-yuexiugongyuan.html', 'name': '广州越秀公园', 'ever': '4972', 'want': '257'}
{'href': '/jingdian-huanghuagang.html', 'name': '广州黄花岗公园', 'ever': '1557', 'want': '189'}
{'href': '/jingdian-

In [3]:
rlt_df = pd.DataFrame(rlt)

NameError: name 'rlt' is not defined

In [ ]:
rlt_df

,name,province,city,location,desc,want,has_been,strategy,route,ticket_info,open_time
0,武汉科技馆,湖北省,武汉市,江岸区,{'景区简介': ['武汉科技馆是一家由武汉市政府兴办，所设展览科技知识含量高，互动性强，融...,223,503,{'最佳游览时间': ['全年']},"{'景区位置': ['中国>湖北省>武汉市>江岸区>塔子湖街道>百步亭花园'], '到达方式...",{'科技馆门票价格': ['成人票：30元/人；天象馆票价：20元/人；4D影院票价：20元...,[9:00-16:30（周二－周日），节假日不休息。]
1,武汉规划展示馆,湖北省,武汉市,江岸区,{'景区简介': ['武汉规划展示馆是市民之家的重要组成部分，全面展示了武汉城市建设的历史、...,28,75,"{'旅游攻略': ['展馆一层综合导览区机器人互动表演时间：10：30、16:00', '展...","{'景区位置': ['中国>湖北省>武汉市>江岸区>塔子湖街道>百步亭花园'], '到达方式...","{'武汉规划展示馆门票价格': ['无门票；', '20人以上团队提供预约免费讲解服务，须提...","[周一至周六9：00-17：00（16：30停止入场）, 周日闭馆维护（法定节日正常开馆）]"
2,武汉古德寺,湖北省,武汉市,江岸区,{'景区简介': ['武汉古德寺依照缅甸阿兰陀寺的艺术形式建造，是一座典型的具有浓郁异域建筑...,98,162,"{'旅游攻略': ['每月农历初一免费'], '最佳游览时间': ['四季皆宜'], '购物...","{'景区位置': ['中国>湖北省>武汉市>江岸区>塔子湖街道>百步亭花园'], '到达方式...",{'古德寺门票价格': ['香火费8元']},[08:00-17:00]
3,武汉二七长江大桥,湖北省,武汉市,江岸区,{'景区简介': ['武汉二七长江大桥是武汉市的第七座长江大桥，也是世界上最大跨度的三塔斜拉...,13,81,{'旅游攻略': ['北端的游玩性更高，可一览大桥风光，休闲娱乐购物美食也很方便。南端更倾向...,"{'景区位置': ['中国>湖北省>武汉市>江岸区>塔子湖街道>百步亭花园'], '到达方式...",{'二七长江大桥门票价格': ['无']},[全天]
4,武汉中山公园,湖北省,武汉市,江汉区,{'景区简介': ['武汉中山公园是武汉市区一座历史悠久的文化休憩公园，被评为全国百家历史名...,105,953,{'最佳游览时间': ['四季皆可']},"{'景区位置': ['中国>湖北省>武汉市>江汉区>江汉经济开发区>武汉中央商务区'], '...",{'中山公园门票价格': ['免费']},[全天]
...,...,...,...,...,...,...,...,...,...,...,...
284,恩施来凤仙佛寺,湖北省,恩施土家族苗族自治州,来凤县,{'景区简介': ['恩施来凤仙佛寺是我国开凿年代最久远的石窟寺之一，也是湖北省唯一的石窑寺...,2,5,"{'最佳游览时间': ['四季皆宜'], '购物推荐': ['油茶汤、零号土猪']}","{'景区位置': ['中国>湖北省>恩施土家族苗族自治州>来凤县'], '到达方式': ['...","{'仙佛寺门票价格': ['80元', '儿童：身高1.2米（含）以下，免费；本票种仅适用于...",[8:30-17:00]
285,鹤峰满山红景区,湖北省,恩施土家族苗族自治州,鹤峰县,{'景区简介': ['鹤峰满山红景区是全国30条红色旅游精品线路的主要景点之一，也是湖北省七...,1,2,{'最佳游览时间': ['四季皆宜']},"{'景区位置': ['中国>湖北省>恩施土家族苗族自治州>鹤峰县'], '到达方式': ['...",{'满山红门票价格': ['免费']},[全天]
286,神农架神农顶旅游区,湖北省,神农架林区,松柏镇,{'景区简介': ['神农架神农顶旅游区位于神农架自然保护区西南部，素有“绿色宝库”、“植物...,3049,1955,"{'旅游攻略': ['神农顶大九湖三日游：', '第一天：燕子垭、天门垭、红坪画廊、香溪源。...","{'景区位置': ['中国>湖北省>神农架林区>松柏镇'], '到达方式': ['从武汉、宜...","{'神农顶门票价格': ['神农顶：130元/人；', '大九湖：100元/人；', '神农...",[6：30-18：30]
287,神农架巴桃园景区,湖北省,神农架林区,松柏镇,{'景区简介': ['巴桃园以“秀、幽、野”而知名，以“清澈、明镜”见常，相传为炎帝果园，因...,1,1,{'最佳游览时间': ['夏秋两季']},"{'景区位置': ['中国>湖北省>神农架林区>松柏镇'], '到达方式': ['麻安高速→...",{'巴桃园门票价格': ['45元，持本地身份证可免门票']},[08:00-17:30]


In [ ]:
rlt_df.to_csv('广东景点.csv', header=True, index=False)